Deleware updates daily at 2pm CDT

In [35]:
from selenium import webdriver
import time
import pandas as pd
import pendulum
import re
import yaml
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=[Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0]")

In [36]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())

In [37]:
state = 'DE'

In [38]:
scrape_timestamp = pendulum.now().strftime('%Y%m%d%H%M%S')

In [39]:
url = 'https://dshs.maps.arcgis.com/apps/opsdashboard/index.html#/b68e95a1ebde4b628c1a776ed4f4bf27'

In [44]:
def fetch():
    driver = webdriver.Chrome('../20190611 - Parts recommendation/chromedriver', options=chrome_options)

    driver.get(url)
    time.sleep(10)
    datatbl = driver.find_elements_by_css_selector('text')
    datatbl.reverse()

    data = []
    for i in range(0, len(datatbl),2):
        data.append([datatbl[i+1].get_attribute('innerHTML'), datatbl[i].get_attribute('innerHTML')])

        if i > 3:
            break
            
    page_source = driver.page_source
    driver.close()
    return pd.DataFrame(data, columns=['county','positive_cases']), page_source

In [41]:
def save(df, source):
    df.to_csv(f"{config['data_folder']}/{state}_county_{scrape_timestamp}.txt", sep='|', index=False)

    with open(f"{config['data_source_backup_folder']}/{state}_county_{scrape_timestamp}.html", 'w') as f:
        f.write(source)

In [42]:
def run():
    df, source = fetch()
    save(df, source)